In [1]:
import numpy as np
import math
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import datasets
import matplotlib.pyplot as plt
train_on_gpu = torch.cuda.is_available()

## Training and Validation Set

In [2]:
def TrainConstructor(path_to_train, path_to_valid, num_workers=4):

    train_transform = transforms.Compose([
        transforms.Resize([224, 224]),
        transforms.ToTensor()
        ])

    train_dataset = datasets.ImageFolder(path_to_train, transform=train_transform)
    print("Successfully Loaded Training Set.")
    valid_dataset = datasets.ImageFolder(path_to_valid, transform=train_transform)
    print("Successfully Loaded Validation Set.")

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, num_workers=num_workers, shuffle=True)
    valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=2, num_workers=num_workers, shuffle=True)

    return train_loader, valid_loader

In [3]:
# train_loader, valid_loader = TrainConstructor(
# path_to_train='C:/Users/fuzha/Downloads/FFT/progan_train/airplane',
# path_to_valid='C:/Users/fuzha/Downloads/FFT/progan_val/airplane', num_workers=1)

## Spec

In [2]:
def conv3x3(in_planes, out_planes, stride=1):
    "3x3 convolution with padding"
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=True)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=True)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=True)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=True)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=2):
        self.inplanes = 64
        # Special attributs
        self.input_space = None
        self.input_size = (299, 299, 3)
        self.mean = None
        self.std = None
        super(ResNet, self).__init__()
        # Modules
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,
                                bias=True)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AvgPool2d(7)
        self.last_linear = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=True),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def features(self, input):
        x = self.conv1(input)
        self.conv1_input = x.clone()
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        return x

    def logits(self, features):
        x = self.avgpool(features)
        x = x.view(x.size(0), -1)
        x = self.last_linear(x)
        return x

    def forward(self, input):
        x = self.features(input)
        x = self.logits(x)
        return x


def ResNet34():
    model = ResNet(BasicBlock, [3, 4, 6, 3], num_classes=2)
    # model.load_state_dict(torch.load(path_to_pretrained))
    return model.cuda()

In [3]:
# model = ResNet34()

# criterion = nn.CrossEntropyLoss()

# optimizer = torch.optim.SGD([
#     {'params': list(model.parameters())[:-1], 'lr': 1e-3, 'momentum': 0.9, 'weight_decay': 1e-3},
#     {'params': list(model.parameters())[-1], 'lr': 5e-5, 'momentum': 0.9, 'weight_decay': 1e-5}
# ])

# optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, dampening=0.9, weight_decay=1e-2)

## Training

In [6]:
train_categories = ['car','cat','horse','person','sofa']

for j in range(5):
    print('Training on category ' + train_categories[j] + ':')
    train_loader, valid_loader = TrainConstructor(path_to_train='E:/Downloads/FFT/progan_train/' + train_categories[j],path_to_valid='E:/Downloads/FFT/progan_val/' + train_categories[j], num_workers=24)

    model = ResNet34()
    model = nn.DataParallel(model)

    # specify loss function (categorical cross-entropy)
    criterion = nn.CrossEntropyLoss()

    # specify optimizer
    optimizer = optimizer = torch.optim.SGD([
        {'params': list(model.parameters())[:-1], 'lr': 1e-3, 'momentum': 0.9, 'weight_decay': 1e-3},
        {'params': list(model.parameters())[-1], 'lr': 5e-5, 'momentum': 0.9, 'weight_decay': 1e-5}
    ])

    # number of epochs to train the model
    n_epochs = 15

    valid_loss_min = np.Inf # track change in validation loss
    training_vis = []
    valid_vis = []

    for epoch in range(1, n_epochs+1):

        # keep track of training and validation loss
        train_loss = 0.0
        valid_loss = 0.0

        ###################
        # train the model #
        ###################
        model.train()
        for data, target in train_loader:
            # move tensors to GPU if CUDA is available
            if train_on_gpu:
                data, target = data.cuda(), target.cuda()

            # clear the gradients of all optimized variables
            optimizer.zero_grad()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            # calculate the batch loss
            loss = criterion(output, target)
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # perform a single optimization step (parameter update)
            optimizer.step()
            # update training loss
            train_loss += loss.item()*data.size(0)

        torch.cuda.empty_cache()    
        ######################    
        # validate the model #
        ######################
        model.eval()
        for data, target in valid_loader:
            # move tensors to GPU if CUDA is available
            if train_on_gpu:
                data, target = data.cuda(), target.cuda()

            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            # calculate the batch loss
            loss = criterion(output, target)
            # update average validation loss 
            valid_loss += loss.item()*data.size(0)

        # calculate average losses
        train_loss = train_loss/len(train_loader.dataset)
        valid_loss = valid_loss/len(valid_loader.dataset)

        training_vis.append(train_loss)
        valid_vis.append(valid_loss)

        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, train_loss, valid_loss))
    #     torch.save(model.state_dict(), 'Xception_' + 'sofa' + '_' + str(epoch) +'.pt')

        # save model if validation loss has decreased
        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...\n'.format(
            valid_loss_min,
            valid_loss))
            torch.save(model.state_dict(), 'Spec_' + train_categories[j] + '_best.pt')
            valid_loss_min = valid_loss

    np.savetxt('Spec_' + train_categories[j] + '.txt', np.array([training_vis, valid_vis]))

    plt.plot(range(epoch), training_vis)
    plt.scatter(range(epoch), training_vis)
    plt.scatter(range(epoch), valid_vis)
    plt.plot(range(epoch), valid_vis)
    plt.savefig('Spec_' + train_categories[j] + '.svg')
    plt.clf()

Training on category car:
Successfully Loaded Training Set.
Successfully Loaded Validation Set.
Epoch: 1 	Training Loss: 0.448443 	Validation Loss: 0.358637
Validation loss decreased (inf --> 0.358637).  Saving model ...

Epoch: 2 	Training Loss: 0.365935 	Validation Loss: 0.296993
Validation loss decreased (0.358637 --> 0.296993).  Saving model ...

Epoch: 3 	Training Loss: 0.332467 	Validation Loss: 0.326469
Epoch: 4 	Training Loss: 0.302007 	Validation Loss: 0.305608
Epoch: 5 	Training Loss: 0.273881 	Validation Loss: 0.353087
Epoch: 6 	Training Loss: 0.242734 	Validation Loss: 0.332347
Epoch: 7 	Training Loss: 0.206556 	Validation Loss: 0.524482
Epoch: 8 	Training Loss: 0.178984 	Validation Loss: 0.508651
Epoch: 9 	Training Loss: 0.146697 	Validation Loss: 0.404421
Epoch: 10 	Training Loss: 0.116248 	Validation Loss: 0.481717
Epoch: 11 	Training Loss: 0.093569 	Validation Loss: 0.669570
Epoch: 12 	Training Loss: 0.084041 	Validation Loss: 0.566511
Epoch: 13 	Training Loss: 0.067767

<Figure size 1200x800 with 0 Axes>

## Load

In [11]:
model = ResNet34()
model = nn.DataParallel(model)
criterion = nn.CrossEntropyLoss()
model.load_state_dict(torch.load('Spec_sofa_best.pt'))

<All keys matched successfully>

## Test

In [3]:
def TestConstructor(path_to_test, classes=None, num_workers=4):

    test_transform = transforms.Compose([
        transforms.Resize([224, 224]),
        transforms.ToTensor()
        ])

    test_dataset = datasets.ImageFolder(path_to_test, transform=test_transform)
    print("Successfully Loaded Test Set.")

    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=2, num_workers=num_workers)

    if classes != None:
        print("Number of Classes:", len(classes))

    return test_loader, classes

In [ ]:
# test_loader, classes = TestConstructor(
# path_to_test='C:\\Users\\fuzha\\Downloads\\CNN_synth_testset\\progan\\airplane',
# classes=['Fake','Real'], num_workers=1)

In [12]:
test_categories = ['airplane','bicycle','bird','boat','bottle','bus','car','cat','chair','cow','diningtable','dog','horse','motorbike','person','pottedplant','sheep','sofa','train','tvmonitor']

for j in range(20):
    print('For test category ' + test_categories[j] + ':')
    test_loader, classes = TestConstructor(path_to_test='C:/Users/fuzha/Downloads/FFT/CNN_synth_testset/progan/' + test_categories[j], classes=['Real','Fake'],num_workers=1)

# test_loader, classes = TestConstructor(path_to_test='C:/Users/fuzha/Downloads/CNN_synth_testset/progan/car', classes=['Real','Fake'], num_workers=1)

# track test loss
    test_loss = 0.0
    class_correct = list(0. for i in range(2))
    class_total = list(0. for i in range(2))

    model.eval()
    # iterate over test data
    for data, target in test_loader:
    #     print(target)
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
    #     print(output)
        # calculate the batch loss
        loss = criterion(output, target)
        # update test loss 
        test_loss += loss.item()*data.size(0)
        # convert output probabilities to predicted class
        _, pred = torch.max(output, 1)
    #     print(pred)
        # compare predictions to true label
        correct_tensor = pred.eq(target.data.view_as(pred))
        correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
        # calculate test accuracy for each object class
        for i in range(2):
    #         print(i)
            label = target.data[i]
            class_correct[label] += correct[i].item()
            class_total[label] += 1

    # average test loss
    test_loss = test_loss/len(test_loader.dataset)
    print('Test Loss: {:.6f}'.format(test_loss))

    for i in range(2):
        if class_total[i] > 0:
            print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
                classes[i], 100 * class_correct[i] / class_total[i],
                np.sum(class_correct[i]), np.sum(class_total[i])))
        else:
            print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

    print('Test Accuracy (Overall): %2d%% (%2d/%2d)\n' % (
        100. * np.sum(class_correct) / np.sum(class_total),
        np.sum(class_correct), np.sum(class_total)))

For test category airplane:
Successfully Loaded Test Set.
Number of Classes: 2
Test Loss: 0.405953
Test Accuracy of  Real: 79% (158/200)
Test Accuracy of  Fake: 84% (169/200)
Test Accuracy (Overall): 81% (327/400)

For test category bicycle:
Successfully Loaded Test Set.
Number of Classes: 2
Test Loss: 0.497440
Test Accuracy of  Real: 85% (170/200)
Test Accuracy of  Fake: 68% (137/200)
Test Accuracy (Overall): 76% (307/400)

For test category bird:
Successfully Loaded Test Set.
Number of Classes: 2
Test Loss: 0.737125
Test Accuracy of  Real: 84% (168/200)
Test Accuracy of  Fake: 48% (97/200)
Test Accuracy (Overall): 66% (265/400)

For test category boat:
Successfully Loaded Test Set.
Number of Classes: 2
Test Loss: 0.560983
Test Accuracy of  Real: 82% (164/200)
Test Accuracy of  Fake: 68% (136/200)
Test Accuracy (Overall): 75% (300/400)

For test category bottle:
Successfully Loaded Test Set.
Number of Classes: 2
Test Loss: 0.345364
Test Accuracy of  Real: 82% (165/200)
Test Accuracy o

In [4]:
train_categories = ['car', 'cat', 'horse', 'person', 'sofa']
test_gans = ['biggan','cyclegan','deepfake','san','stargan','stylegan','stylegan2','whichfaceisreal']
cyclegan_categories = ['apple','horse','orange','summer','winter','zebra']
stylegan_categories = ['bedroom','car','cat']
stylegan2_categories = ['car','cat','church','horse']

for train_category in train_categories:
    print('For train category ' + train_category + ':')
    model = ResNet34()
    model = nn.DataParallel(model)
    criterion = nn.CrossEntropyLoss()
    model.load_state_dict(torch.load('Spec_' + train_category + '_best.pt'))

    for test_gan in test_gans:
        print('For test gan ' + test_gan + ':')
        test_categories = []
        if test_gan == 'cyclegan':
            test_categories = cyclegan_categories
        elif test_gan == 'stylegan':
            test_categories = stylegan_categories
        elif test_gan == 'stylegan2':
            test_categories = stylegan2_categories

        if len(test_categories) == 0:
            test_loader, classes = TestConstructor(path_to_test='C:/Users/fuzha/Downloads/FFT/CNN_synth_testset/' + test_gan, classes=['Real','Fake'], num_workers=1)

            # track test loss
            test_loss = 0.0
            class_correct = list(0. for i in range(2))
            class_total = list(0. for i in range(2))

            model.eval()
            # iterate over test data
            for data, target in test_loader:
            #     print(target)
                # move tensors to GPU if CUDA is available
                if train_on_gpu:
                    data, target = data.cuda(), target.cuda()
                # forward pass: compute predicted outputs by passing inputs to the model
                output = model(data)
            #     print(output)
                # calculate the batch loss
                loss = criterion(output, target)
                # update test loss 
                test_loss += loss.item()*data.size(0)
                # convert output probabilities to predicted class
                _, pred = torch.max(output, 1)
            #     print(pred)
                # compare predictions to true label
                correct_tensor = pred.eq(target.data.view_as(pred))
                correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
                # calculate test accuracy for each object class
                for i in range(2):
            #         print(i)
                    label = target.data[i]
                    class_correct[label] += correct[i].item()
                    class_total[label] += 1

            # average test loss
            test_loss = test_loss/len(test_loader.dataset)
            print('Test Loss: {:.6f}'.format(test_loss))

            for i in range(2):
                if class_total[i] > 0:
                    print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
                        classes[i], 100 * class_correct[i] / class_total[i],
                        np.sum(class_correct[i]), np.sum(class_total[i])))
                else:
                    print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

            print('Test Accuracy (Overall): %2d%% (%2d/%2d)\n' % (
                100. * np.sum(class_correct) / np.sum(class_total),
                np.sum(class_correct), np.sum(class_total)))

        else:
            for test_category in test_categories:
                print('For test category ' + test_category + ':')
                test_loader, classes = TestConstructor(path_to_test='C:/Users/fuzha/Downloads/FFT/CNN_synth_testset/' + test_gan + '/' + test_category, classes=['Real','Fake'], num_workers=1)

                # track test loss
                test_loss = 0.0
                class_correct = list(0. for i in range(2))
                class_total = list(0. for i in range(2))

                model.eval()
                # iterate over test data
                for data, target in test_loader:
                #     print(target)
                    # move tensors to GPU if CUDA is available
                    if train_on_gpu:
                        data, target = data.cuda(), target.cuda()
                    # forward pass: compute predicted outputs by passing inputs to the model
                    output = model(data)
                #     print(output)
                    # calculate the batch loss
                    loss = criterion(output, target)
                    # update test loss 
                    test_loss += loss.item()*data.size(0)
                    # convert output probabilities to predicted class
                    _, pred = torch.max(output, 1)
                #     print(pred)
                    # compare predictions to true label
                    correct_tensor = pred.eq(target.data.view_as(pred))
                    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
                    # calculate test accuracy for each object class
                    for i in range(2):
                #         print(i)
                        label = target.data[i]
                        class_correct[label] += correct[i].item()
                        class_total[label] += 1

                # average test loss
                test_loss = test_loss/len(test_loader.dataset)
                print('Test Loss: {:.6f}'.format(test_loss))

                for i in range(2):
                    if class_total[i] > 0:
                        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
                            classes[i], 100 * class_correct[i] / class_total[i],
                            np.sum(class_correct[i]), np.sum(class_total[i])))
                    else:
                        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

                print('Test Accuracy (Overall): %2d%% (%2d/%2d)\n' % (
                    100. * np.sum(class_correct) / np.sum(class_total),
                    np.sum(class_correct), np.sum(class_total)))

For train category car:
For test gan biggan:
Successfully Loaded Test Set.
Number of Classes: 2
Test Loss: 1.899005
Test Accuracy of  Real: 43% (869/2000)
Test Accuracy of  Fake: 34% (687/2000)
Test Accuracy (Overall): 38% (1556/4000)

For test gan cyclegan:
For test category apple:
Successfully Loaded Test Set.
Number of Classes: 2
Test Loss: 1.666133
Test Accuracy of  Real: 53% (142/266)
Test Accuracy of  Fake: 28% (71/248)
Test Accuracy (Overall): 41% (213/514)

For test category horse:
Successfully Loaded Test Set.
Number of Classes: 2
Test Loss: 1.290696
Test Accuracy of  Real: 63% (76/120)
Test Accuracy of  Fake: 30% (43/140)
Test Accuracy (Overall): 45% (119/260)

For test category orange:
Successfully Loaded Test Set.
Number of Classes: 2
Test Loss: 2.304403
Test Accuracy of  Real: 52% (129/248)
Test Accuracy of  Fake:  7% (20/266)
Test Accuracy (Overall): 28% (149/514)

For test category summer:
Successfully Loaded Test Set.
Number of Classes: 2
Test Loss: 1.124066
Test Accura